In [1]:
import os
import random 
from glob import glob 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
import pandas as pd

2022-02-11 13:27:51.090827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-11 13:27:51.090873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

class SpeechFeatureEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)


In [3]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [4]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm


In [5]:
class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_input = SpeechFeatureEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input


In [6]:
audio = "/home/divyansh/Divyansh/projects/senti_random_forest/data/"
actor_folders = os.listdir(audio)
actor_folders.sort()
actor_folders[0:5]

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05']

In [7]:
emotion = []
gender = []
actor = []
file_path = []
for i in actor_folders:
    filename = os.listdir(audio + i) #iterate over Actor folders
    for f in filename: # go through files in Actor folder
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        actor.append(int(part[6]))
        bg = int(part[6])
        if bg%2 == 0:
            bg = "female"
        else:
            bg = "male"
        gender.append(bg)
        file_path.append(audio + i + '/' + f)

In [8]:
audio_df = pd.DataFrame(emotion)
audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['path'])],axis=1)
audio_df

,gender,emotion,actor,path
0,male,neutral,1,/home/divyansh/Divyansh/projects/senti_random_...
1,male,fear,1,/home/divyansh/Divyansh/projects/senti_random_...
2,male,angry,1,/home/divyansh/Divyansh/projects/senti_random_...
3,male,happy,1,/home/divyansh/Divyansh/projects/senti_random_...
4,male,sad,1,/home/divyansh/Divyansh/projects/senti_random_...
...,...,...,...,...
1435,female,sad,24,/home/divyansh/Divyansh/projects/senti_random_...
1436,female,fear,24,/home/divyansh/Divyansh/projects/senti_random_...
1437,female,calm,24,/home/divyansh/Divyansh/projects/senti_random_...
1438,female,surprise,24,/home/divyansh/Divyansh/projects/senti_random_...


In [9]:
saveto = "./dataset/LJSpeech-1.1/"
wavs = glob("{}/**/*.wav".format(saveto), recursive=True)

In [10]:
saveto

'./dataset/LJSpeech-1.1/'

In [11]:
wavs

['./dataset/LJSpeech-1.1/wavs/LJ008-0155.wav',
 './dataset/LJSpeech-1.1/wavs/LJ014-0143.wav',
 './dataset/LJSpeech-1.1/wavs/LJ003-0182.wav',
 './dataset/LJSpeech-1.1/wavs/LJ048-0168.wav',
 './dataset/LJSpeech-1.1/wavs/LJ003-0008.wav',
 './dataset/LJSpeech-1.1/wavs/LJ040-0115.wav',
 './dataset/LJSpeech-1.1/wavs/LJ038-0005.wav',
 './dataset/LJSpeech-1.1/wavs/LJ039-0043.wav',
 './dataset/LJSpeech-1.1/wavs/LJ033-0053.wav',
 './dataset/LJSpeech-1.1/wavs/LJ002-0119.wav',
 './dataset/LJSpeech-1.1/wavs/LJ003-0077.wav',
 './dataset/LJSpeech-1.1/wavs/LJ038-0267.wav',
 './dataset/LJSpeech-1.1/wavs/LJ011-0040.wav',
 './dataset/LJSpeech-1.1/wavs/LJ040-0004.wav',
 './dataset/LJSpeech-1.1/wavs/LJ020-0016.wav',
 './dataset/LJSpeech-1.1/wavs/LJ013-0150.wav',
 './dataset/LJSpeech-1.1/wavs/LJ043-0092.wav',
 './dataset/LJSpeech-1.1/wavs/LJ035-0185.wav',
 './dataset/LJSpeech-1.1/wavs/LJ050-0192.wav',
 './dataset/LJSpeech-1.1/wavs/LJ039-0013.wav',
 './dataset/LJSpeech-1.1/wavs/LJ049-0056.wav',
 './dataset/L

In [12]:
id_to_text = {}
with open(os.path.join(saveto,"metadata.csv"), encoding="utf-8") as f:
    for line in f:
        id = line.strip().split("|")[0]
        text = line.strip().split("|")[2]
        id_to_text[id] = text



In [13]:
def get_data(wavs, id_to_text, maxlen = 50):
    data = []
    for w in wavs:
        id = w.split("/")[-1].split(".")[0]
        if len(id_to_text[id])< maxlen:
            data.append({"audio": w, "text": id_to_text[id]})
    return data

In [14]:
#Pre - PROCESS THE DATA 
#VECTORIZE THE DATA 

class VectorizeChar:
    def __init__(self,max_len = 50):
        self.vocab = ( ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
                     )
        self.max_len = max_len
        self.char_to_idx = {}
        for i,ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i
    
    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab

In [15]:
max_target_len = 200
data = get_data(wavs, id_to_text, max_target_len)


In [16]:
data

[{'audio': './dataset/LJSpeech-1.1/wavs/LJ008-0155.wav',
  'text': 'Very great excitement prevailed in the town throughout the trial, and this greatly increased when the verdict was known.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ014-0143.wav',
  'text': 'On being removed to Newgate from the court Mrs. Manning became perfectly furious.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ003-0182.wav',
  'text': 'The tried and the untried, young and old, were herded together'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ048-0168.wav',
  'text': 'With the number of men available to the Secret Service and the time available, surveys of hundreds of buildings and thousands of windows is not practical.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ003-0008.wav',
  'text': "Friends arrived about nine a.m., and went out at nine p.m., when as many as two hundred visitors have been observed leaving the debtors' yards at one time."},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ040-0115.wav',
  'text': '

In [17]:
vetorizer = VectorizeChar(max_target_len)

In [18]:
vetorizer

In [19]:
print("vocab size", len(vetorizer.get_vocabulary()))

vocab size 34


In [20]:
def create_text_ds(data):
    texts = [_["text"] for _ in data]
    text_ds = [vetorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds

In [21]:
def path_to_audio(path):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)
    # normalisation
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = (x - means) / stddevs
    audio_len = tf.shape(x)[0]
    # padding to 10 seconds
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x


In [22]:
def create_audio_ds(data):
    flist = [_["audio"] for _ in data]
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(
        path_to_audio, num_parallel_calls=tf.data.AUTOTUNE
    )
    return audio_ds

In [23]:
def create_tf_dataset(data, bs=4):
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [24]:
split = int(len(data)* 0.99)
train_data = data[:split]
test_Data = data[split:]
ds = create_tf_dataset(train_data,bs=64)
val_ds = create_tf_dataset(test_Data, bs=4)

2022-02-11 13:27:54.658706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-11 13:27:54.658764: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-11 13:27:54.658789: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (divyansh-Lenovo-V14-ARE): /proc/driver/nvidia/version does not exist
2022-02-11 13:27:54.659567: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
ds

<PrefetchDataset shapes: {source: (None, None, 129), target: (None, 200)}, types: {source: tf.float32, target: tf.int32}>

In [26]:
val_ds

<PrefetchDataset shapes: {source: (None, None, 129), target: (None, 200)}, types: {source: tf.float32, target: tf.int32}>

In [27]:
train_data

[{'audio': './dataset/LJSpeech-1.1/wavs/LJ008-0155.wav',
  'text': 'Very great excitement prevailed in the town throughout the trial, and this greatly increased when the verdict was known.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ014-0143.wav',
  'text': 'On being removed to Newgate from the court Mrs. Manning became perfectly furious.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ003-0182.wav',
  'text': 'The tried and the untried, young and old, were herded together'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ048-0168.wav',
  'text': 'With the number of men available to the Secret Service and the time available, surveys of hundreds of buildings and thousands of windows is not practical.'},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ003-0008.wav',
  'text': "Friends arrived about nine a.m., and went out at nine p.m., when as many as two hundred visitors have been observed leaving the debtors' yards at one time."},
 {'audio': './dataset/LJSpeech-1.1/wavs/LJ040-0115.wav',
  'text': '

In [28]:
class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Displays a batch of outputs after every epoch

        Args:
            batch: A test batch containing the keys "source" and "target"
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")


In [29]:
class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """ linear warm up - linear decay """
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / (self.decay_epochs),
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        return self.calculate_lr(epoch)

In [30]:
batch = next(iter(val_ds))

# The vocabulary to convert predicted indices into characters
idx_to_char = vetorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)  # set the arguments as per vocabulary index for '<' and '>'

model = Transformer(
    num_hid=200,
    num_head=2,
    num_feed_forward=400,
    target_maxlen=max_target_len,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=34,
)
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)

learning_rate = CustomSchedule(
    init_lr=0.00001,
    lr_after_warmup=0.001,
    final_lr=0.00001,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(ds),
)
optimizer = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss=loss_fn)

history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=1)


203/203 [==============================] - ETA: 0s - loss: 1.7298target:     <white managed to keep his wrists asunder, and continued to struggle with the officials for some time. eventually he was pinioned with a cord in the usual manner.>
prediction: <the on taont at in at it catare t t one o on in in ir if ant in he ar ar atah t tt it t t t int an e intind otl t o te at in s inin t int a ilthe the s it re onthnee aheon the anin ard or cane at ca 

target:     <it would have been possible for him to take different routes in approaching and leaving the scene of the shooting.>
prediction: <the on taont at in at it catare t t one o on in in ir if ant in he ar ar atah t tt it t t t int an e intind otl t t te at in s inin t int a ilthe the s ithe oonthnee aheon the anin ard or cane at ca 

target:     <in giltspur street compter, where he was first lodged>
prediction: <the on taont at in at it catare t the s o on in ine r if ant in he ar ar atah t tt it t t t int an e intind otl t it onnt